In [18]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps

# Import the random module.
import random

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.
import requests

# Import the API key and G Key.
from config import weather_api_key
from config import g_key

# Import the datetime module from the datetime library.
from datetime import datetime

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

In [19]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches,Snow Inches
0,0,Puerto Ayora,EC,-0.74,-90.35,70.00,86,88,1.99,overcast clouds,0.00,0
1,1,Ancud,CL,-41.87,-73.82,46.99,81,20,4.70,few clouds,0.00,0
2,2,Shitanjing,CN,39.23,106.34,66.15,70,0,3.71,clear sky,0.00,0
3,3,Fuling,CN,29.70,107.39,75.58,93,100,1.50,moderate rain,1.44,0
4,4,Topeka,US,39.05,-95.68,87.80,43,1,14.99,clear sky,0.00,0


In [20]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preference=str(input("Do you want it to be raining? (yes/no) "))
snow_preference=str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [21]:
# Filter on no rain and no snow
if rain_preference == "no" and snow_preference == "no":
    
# Filter the dataset to find the cities that fit the criteria.
    city_and_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& (city_data_df["Snow Inches"] == 0) & (city_data_df["Rain Inches"] == 0) ]

In [22]:
# Filter on yes rain and no snow
if rain_preference == "yes" and snow_preference == "no":
    
# Filter the dataset to find the cities that fit the criteria.
    city_and_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& (city_data_df["Snow Inches"] == 0) & (city_data_df["Rain Inches"] > 0) ]

In [23]:
# Filter on no rain and yes snow
if rain_preference == "no" and snow_preference == "yes":
    
# Filter the dataset to find the cities that fit the criteria.
    city_and_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& (city_data_df["Snow Inches"] > 0) & (city_data_df["Rain Inches"] == 0) ]
city_and_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches,Snow Inches
0,0,Puerto Ayora,EC,-0.74,-90.35,70.00,86,88,1.99,overcast clouds,0.0,0
2,2,Shitanjing,CN,39.23,106.34,66.15,70,0,3.71,clear sky,0.0,0
6,6,Hasaki,JP,35.73,140.83,64.40,100,75,6.93,broken clouds,0.0,0
8,8,Avarua,CK,-21.21,-159.78,69.80,94,94,6.93,heavy intensity rain,0.0,0
9,9,Jamestown,US,42.10,-79.24,75.00,60,1,12.75,clear sky,0.0,0


In [24]:
# Filter on no rain and yes snow
if rain_preference == "yes" and snow_preference == "yes":
    
# Filter the dataset to find the cities that fit the criteria.
    city_and_weather_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)& (city_data_df["Snow Inches"] > 0) & (city_data_df["Rain Inches"] > 0) ]
city_and_weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches,Snow Inches
0,0,Puerto Ayora,EC,-0.74,-90.35,70.00,86,88,1.99,overcast clouds,0.0,0
2,2,Shitanjing,CN,39.23,106.34,66.15,70,0,3.71,clear sky,0.0,0
6,6,Hasaki,JP,35.73,140.83,64.40,100,75,6.93,broken clouds,0.0,0
8,8,Avarua,CK,-21.21,-159.78,69.80,94,94,6.93,heavy intensity rain,0.0,0
9,9,Jamestown,US,42.10,-79.24,75.00,60,1,12.75,clear sky,0.0,0


In [25]:
city_and_weather_df.count()

City_ID                298
City                   298
Country                296
Lat                    298
Lng                    298
Max Temp               298
Humidity               298
Cloudiness             298
Wind Speed             298
Current Description    298
Rain Inches            298
Snow Inches            298
dtype: int64

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_and_weather_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,70.00,overcast clouds,-0.74,-90.35,
2,Shitanjing,CN,66.15,clear sky,39.23,106.34,
6,Hasaki,JP,64.40,broken clouds,35.73,140.83,
8,Avarua,CK,69.80,heavy intensity rain,-21.21,-159.78,
9,Jamestown,US,75.00,clear sky,42.10,-79.24,
10,Poykovskiy,RU,57.20,broken clouds,61.23,73.33,
12,Lebu,CL,50.36,overcast clouds,-37.62,-73.65,
13,Ondangwa,NaN,64.40,clear sky,-17.92,15.95,
22,Gweta,BW,50.36,clear sky,-20.18,25.23,
23,Bubaque,GW,79.02,overcast clouds,11.28,-15.83,


In [27]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [34]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found... skipping.")

In [29]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,70.00,overcast clouds,-0.74,-90.35,
2,Shitanjing,CN,66.15,clear sky,39.23,106.34,
6,Hasaki,JP,64.40,broken clouds,35.73,140.83,
8,Avarua,CK,69.80,heavy intensity rain,-21.21,-159.78,
9,Jamestown,US,75.00,clear sky,42.10,-79.24,
10,Poykovskiy,RU,57.20,broken clouds,61.23,73.33,
12,Lebu,CL,50.36,overcast clouds,-37.62,-73.65,
13,Ondangwa,NaN,64.40,clear sky,-17.92,15.95,
22,Gweta,BW,50.36,clear sky,-20.18,25.23,
23,Bubaque,GW,79.02,overcast clouds,11.28,-15.83,


In [30]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the hotel_df into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")